In [38]:
import numpy as np
import unittest
import heapq as hq
import weakref

In [39]:
def numerical_diff(f, x, eps=1e-4):
    return (f(Variable(x.data+eps)).data - f(Variable(x.data-eps)).data) / (2*eps)
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [40]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}는 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def __str__(self):
        return f'{self.generation} gen : [{__class__}], {id(self)}'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        seen_set = set()
        funcs = []

        def add_func(f):
            if f not in seen_set:
                seen_set.add(f)
                hq.heappush(funcs, f)
        
        add_func(self.creator)

        while funcs:
            f = hq.heappop(funcs)
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx if x.grad is None else gx + x.grad
                if x.creator is not None:
                    add_func(x.creator)

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys, )
        self.generation = max([x.generation for x in inputs])
        outputs = [Variable(as_array(y)) for y in ys]
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other): # for generation heap
        if self.generation > other.generation:
            return True
        return False

    def __str__(self):
        return f'{self.generation} gen : [{__class__}], {id(self)}'

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, x):
        raise NotImplementedError()

class Square(Function):
    def forward(self, x):
        return x**2
    def backward(self, gy):
        x = self.inputs[0].data
        return 2 * x * gy

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    def backward(self, gy):
        return np.exp(self.inputs.data) * gy

class Add(Function):
    def forward(self, x0, x1):
        return (x0 + x1, )
    def backward(self, gy):
        return gy, gy

In [41]:
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6)
        self.assertEqual(x.grad, expected)
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

In [42]:
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)
def add(x, y):
    return Add()(x, y)

In [43]:
print(id(hq))
d = hq.heapify([])
print(id(d))

139891078027280
9484816


In [44]:
x = Variable(np.array(2.0))
a = square(x)
b = square(a)
c = square(a)
y = add(b, c)
y.backward()
print(y.data, x.grad, a.grad)
print(x, a, b, c, y)

32.0 64.0 16.0
0 gen : [<class '__main__.Variable'>], 139890257604672 1 gen : [<class '__main__.Variable'>], 139890257607120 2 gen : [<class '__main__.Variable'>], 139890257608368 2 gen : [<class '__main__.Variable'>], 139890274310080 3 gen : [<class '__main__.Variable'>], 139890274311808


In [45]:
!pip install memory_profiler

     |████████████████████████████████| 296 kB 1.0 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30182 sha256=8d67898d0999a1becc0a6e3eab7f1a2c84562ddb8879a4957be4db6654697d09
  Stored in directory: /home/cod3holic/.cache/pip/wheels/6a/37/3e/d9e8ebaf73956a3ebd2ee41869444dbd2a702d7142bcf93c42
Successfully built memory-profiler
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
